In [1]:
# Import required libraries
import chess
import chess.engine
import chess.pgn
import chess.svg
from cairosvg import svg2png
import subprocess
import os


OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': dlopen(libcairo.so.2, 0x0002): tried: 'libcairo.so.2' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibcairo.so.2' (no such file), '/Users/alankritverma/miniconda3/envs/CSSLAB/bin/../lib/libcairo.so.2' (no such file), '/usr/lib/libcairo.so.2' (no such file, not in dyld cache), 'libcairo.so.2' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libcairo.so.2'
cannot load library 'libcairo.2.dylib': dlopen(libcairo.2.dylib, 0x0002): tried: 'libcairo.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibcairo.2.dylib' (no such file), '/Users/alankritverma/miniconda3/envs/CSSLAB/bin/../lib/libcairo.2.dylib' (no such file), '/usr/lib/libcairo.2.dylib' (no such file, not in dyld cache), 'libcairo.2.dylib' (no such file).  Additionally, ctypes.util.find_library() did n

In [ ]:
# ## Paths to Engines and Models

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}


In [ ]:
# ## Initialize Engines

# Initialize Stockfish engine
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path, stderr=subprocess.DEVNULL)

# Initialize Maia engines
maia_engines = {
    elo: chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"], stderr=subprocess.DEVNULL)
    for elo, path in maia_model_paths.items()
}


In [ ]:
# ## Load and Analyze Games

def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

def analyze_with_engines(game, stockfish_engine, maia_engines):
    board = game.board()
    blunders = []

    for move in game.mainline_moves():
        board.push(move)

        # Analyze with Stockfish
        stockfish_info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1))
        stockfish_eval = stockfish_info["score"].relative.score(mate_score=10000) / 100.0
        stockfish_best_move = stockfish_info["pv"][0] if "pv" in stockfish_info else None

        # Define blunder threshold
        if abs(stockfish_eval) > 1.5:
            maia_suggestions = {}
            for elo, maia_engine in maia_engines.items():
                maia_info = maia_engine.analyse(board, chess.engine.Limit(nodes=1))
                maia_eval = maia_info["score"].relative.score(mate_score=10000) / 100.0
                maia_best_move = maia_info["pv"][0] if "pv" in maia_info else None
                maia_suggestions[elo] = (maia_best_move, maia_eval)

            blunders.append((board.fen(), move, stockfish_eval, stockfish_best_move, maia_suggestions))

    return blunders

def filter_best_blunders(blunders):
    filtered_blunders = []
    for fen, move, stockfish_eval, stockfish_best_move, maia_suggestions in blunders:
        consistent_moves = all(
            maia_suggestions[elo][0] == stockfish_best_move
            for elo in maia_suggestions if elo > 1300
        )
        if consistent_moves:
            filtered_blunders.append((fen, move, stockfish_eval, stockfish_best_move, maia_suggestions))
    return filtered_blunders

def output_blunder_analysis(blunders):
    for fen, move, stockfish_eval, stockfish_best_move, maia_suggestions in blunders:
        print(f"Position: {fen}")
        print(f"Blunder: {move}")
        print(f"Stockfish Evaluation: {stockfish_eval}")
        print(f"Stockfish Suggested Move: {stockfish_best_move}")
        for elo, (best_move, maia_eval) in maia_suggestions.items():
            print(f"Maia {elo} Evaluation: {maia_eval}, Suggested Move: {best_move}")
        print()

# Load games from PGN file
pgn_file_path = "analysis_pgns/0KNdpNQd---1910 Lasker vs. Schlechter.pgn"
all_games = load_games_from_pgn(pgn_file_path)

# Analyze games
all_blunders = []
for game in all_games:
    blunders = analyze_with_engines(game, stockfish_engine, maia_engines)
    filtered_blunders = filter_best_blunders(blunders)
    if filtered_blunders:
        all_blunders.append((game.headers["Event"], filtered_blunders))

# Output analysis
for event, blunder_list in all_blunders:
    print(f"Event: {event}")
    output_blunder_analysis(blunder_list)


In [ ]:
# ## Generate Images for Critical Positions

# Ensure you have cairosvg installed:
# pip install cairosvg

# Function to save board image with annotations
def save_board_image(board, move=None, output_file="board.png"):
    svg_data = chess.svg.board(board, lastmove=move)
    svg2png(bytestring=svg_data, write_to=output_file)

# Function to highlight a move on the board
def highlight_move(board, move, output_file="highlighted_board.png"):
    board.push(move)
    save_board_image(board, move, output_file)
    board.pop()  # Revert the move

# Directory to save images
image_dir = "chess_images"
os.makedirs(image_dir, exist_ok=True)

# Generate images for each critical position
for event, blunder_list in all_blunders:
    print(f"Event: {event}")
    for i, (fen, move, stockfish_eval, stockfish_best_move, maia_suggestions) in enumerate(blunder_list):
        board = chess.Board(fen)

        # Save initial board state
        initial_image_path = os.path.join(image_dir, f"{event.replace(' ', '_')}_pos_{i}_initial.png")
        save_board_image(board, output_file=initial_image_path)

        # Highlight the blunder move
        blunder_image_path = os.path.join(image_dir, f"{event.replace(' ', '_')}_pos_{i}_blunder.png")
        highlight_move(board, move, output_file=blunder_image_path)

        # Highlight the correct move
        correct_image_path = os.path.join(image_dir, f"{event.replace(' ', '_')}_pos_{i}_correct.png")
        highlight_move(board, stockfish_best_move, output_file=correct_image_path)

        print(f"Position: {fen}")
        print(f"Blunder: {move}")
        print(f"Stockfish Evaluation: {stockfish_eval}")
        print(f"Stockfish Suggested Move: {stockfish_best_move}")
        for elo, (best_move, maia_eval) in maia_suggestions.items():
            print(f"Maia {elo} Evaluation: {maia_eval}, Suggested Move: {best_move}")
        print(f"Initial Image: {initial_image_path}")
        print(f"Blunder Image: {blunder_image_path}")
        print(f"Correct Move Image: {correct_image_path}")
        print()


In [ ]:
# ## Cleanup Engines

# Cleanup
stockfish_engine.quit()
for maia_engine in maia_engines.values():
    maia_engine.quit()